In [27]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import os
import cv2

#To run the prediction on the GPU, else comment out this line to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import scipy.misc
from csbdeep.utils.tf import limit_gpu_memory
from skimage.filters import threshold_otsu
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.path.append('../../Terminator/')
import imageio
from IPython.display import clear_output
import glob
from tifffile import imread
import tqdm
from csbdeep.utils import Path, normalize
from stardist import random_label_cmap, _draw_polygons,dist_to_coord
from stardist.models import StarDist2D
from TerminatorUtils.helpers import normalizeFloat, OtsuThreshold2D, save_8bit_tiff_imagej_compatible, save_16bit_tiff_imagej_compatible,Integer_to_border,SeedStarDistWatershed
from TerminatorUtils.helpers import Prob_to_Binary, zero_pad, SeedStarDistWatershedV2
from TerminatorUtils.helpers import save_tiff_imagej_compatible
from csbdeep.models import Config, CARE
np.random.seed(6)
lbl_cmap = random_label_cmap()

from skimage.morphology import remove_small_objects
from skimage.morphology import skeletonize
import png

import time

#from TerminatorUtils.helpers import limit_gpu_memory_actualised
#limit_gpu_memory_actualised(0.25)


In [18]:
cutdir = ''
basedir = '/Cut/'
basedir_StardistWater_FijiMask = '/StarWat_FijiMask/'
basedir_StardistWater_FijiMask_RTGPipeline = '/StarWat_FijiMask_RTGPipeline/'

Model_Dir = '/home/sancere/NextonDisk_1/Lucas_Model_to_use/Segmentation/'  

In [22]:
Path(basedir).mkdir(exist_ok = True)
Raw_path = os.path.join(cutdir, '*tif') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw = glob.glob(Raw_path)
filesRaw.sort


for fname in filesRaw:
   
    BigImage = imread(fname)
    SmallImageA = BigImage[0:BigImage.shape[0]//2,0:BigImage.shape[1]//2 ]
    SmallImageB = BigImage[BigImage.shape[0]//2:BigImage.shape[0],0:BigImage.shape[1]//2 ]
    SmallImageC = BigImage[0:BigImage.shape[0]//2,BigImage.shape[1]//2 :BigImage.shape[1]]
    SmallImageD = BigImage[BigImage.shape[0]//2:BigImage.shape[0],BigImage.shape[1]//2 :BigImage.shape[1]]
    
    HomeMadeMask = cv2.imread(fname.replace('.tif','_Mask.png'))
    HomeMadeMaskA = HomeMadeMask[0:HomeMadeMask.shape[0]//2,0:HomeMadeMask.shape[1]//2 ]
    HomeMadeMaskB = HomeMadeMask[HomeMadeMask.shape[0]//2:HomeMadeMask.shape[0],0:HomeMadeMask.shape[1]//2 ]
    HomeMadeMaskC = HomeMadeMask[0:HomeMadeMask.shape[0]//2,HomeMadeMask.shape[1]//2 :HomeMadeMask.shape[1]]
    HomeMadeMaskD = HomeMadeMask[HomeMadeMask.shape[0]//2:HomeMadeMask.shape[0],HomeMadeMask.shape[1]//2 :HomeMadeMask.shape[1]]
    
    save_tiff_imagej_compatible((basedir + 'P1_' + os.path.basename(os.path.splitext(fname)[0]) + '.tif') , SmallImageA , axes)
    imageio.imwrite(basedir + 'P1_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMaskA)
    save_tiff_imagej_compatible((basedir + 'P2_' + os.path.basename(os.path.splitext(fname)[0])+ '.tif' ) , SmallImageB , axes)
    imageio.imwrite(basedir + 'P2_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMaskB) 
    save_tiff_imagej_compatible((basedir + 'P3_' + os.path.basename(os.path.splitext(fname)[0])+ '.tif' ) , SmallImageC , axes)
    imageio.imwrite(basedir + 'P3_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMaskC)
    save_tiff_imagej_compatible((basedir + 'P4_' + os.path.basename(os.path.splitext(fname)[0])+ '.tif' ) , SmallImageD , axes)
    imageio.imwrite(basedir + 'P4_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMaskD)
    

In [23]:
StardistModelName = 'DrosophilaSegmentationSmartSeedsMatureMacroMiniElongatedCyc05LargePatch'

modelstar = StarDist2D(config = None, name = StardistModelName, basedir = Model_Dir)








Loading network weights from 'weights_best.h5'.






Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.


In [24]:
Path(basedir_StardistWater_FijiMask).mkdir(exist_ok = True)
Raw_path = os.path.join(basedir, '*tif') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw = glob.glob(Raw_path)
filesRaw.sort

<function list.sort>

In [30]:
for fname in filesRaw:
 
    if os.path.exists((basedir_StardistWater_FijiMask + 'StarWat_FijiMask_' + os.path.basename(os.path.splitext(fname)[0]))) == False :
        
            # Name = os.path.basename(os.path.splitext(fname)[0])
            x = imread(fname)
            y = normalizeFloat(x,1,99.8, axis = axis_norm)
            originalX = x.shape[0]
            originalY = x.shape[1]
        
            #zero pad (Never do 8bit conversion when applying prediction on an image, only float32)
               y = zero_pad(y, 32, 32)
          
            print('Processing file', fname)            
            
         
            #Stardist, label image, details, probability map, distance map 
            segmentationImage, details = modelstar.predict_instances(y, n_tiles = (3,3))
            prob, dist = modelstar.predict(y, n_tiles = (3, 3))
            grid = modelstar.config.grid
            prob = cv2.resize(prob,  dsize=(prob.shape[1] * grid[1],prob.shape[0] * grid[0]))
            
      


            #Seeds from Stardist + Whatershed, segmentation on probability map 
            Watershed, markers= SeedStarDistWatershedV2(prob,segmentationImage,modelstar.config.grid) 
            
            #Convert Integer image to binary
            Binary_Watershed = Integer_to_border(Watershed[:originalX, :originalY])
           
            #Generation of masks                             
            HomeMadeMask = cv2.imread(fname.replace('.tif','.png'))
            HomeMadeMask = HomeMadeMask[:,:,0]
            print(Binary_Watershed.shape,HomeMadeMask.shape )

            #Create StardistWater_StardistMask and StardistWater_UnetMask with logical and operation            
            StardistWater_FijiMask = np.logical_and(HomeMadeMask, Binary_Watershed[:originalX, :originalY])


            #Skeletonization and deletion of non closed cells. Has to be done AFTER logical and operation 
            z = StardistWater_FijiMask 
            z = skeletonize(z) 
            z = np.float32(z)
            z2 = z.copy()
            mask = np.zeros((np.array(z.shape)+2), np.uint8)
            cv2.floodFill(z, mask, (0,0), (255))
            z = cv2.erode(z, np.ones((3,3)))
            z = cv2.bitwise_not(z)
            z = cv2.bitwise_and(z,z2)
            StardistWater_FijiMask = z 

            #Save different method segmentations
            save_8bit_tiff_imagej_compatible((basedir_StardistWater_FijiMask + 'StarWat_FijiMask_' + os.path.basename(os.path.splitext(fname)[0]) ) , StardistWater_FijiMask, axes)


  0%|          | 0/9 [00:00<?, ?it/s]

Processing file /home/sancere/NextonDisk_1/FromYo2Lucas/Broken/0Cadherine_Movie_Jesus.tif


100%|██████████| 9/9 [00:10<00:00,  1.12s/it]


(3623, 3582) (3623, 3582)


  0%|          | 0/9 [00:00<?, ?it/s]

...

(3623, 3583) (3623, 3583)


  0%|          | 0/9 [00:00<?, ?it/s]

Processing file /home/sancere/NextonDisk_1/FromYo2Lucas/Broken/3Cadherine_Movie_Jesus.tif


100%|██████████| 9/9 [00:09<00:00,  1.10s/it]


(3623, 3583) (3623, 3583)


In [ ]:

import png

Path(basedir_StardistWater_FijiMask_RTGPipeline).mkdir(exist_ok = True)

Raw_path_2 = os.path.join(basedir_StardistWater_FijiMask, '*') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw_2 = glob.glob(Raw_path_2)

    
for fname in filesRaw_2:
    z = imread(fname)
    z = 255 * z
    z = cv2.bitwise_not(z)
    name = str(os.path.basename(os.path.splitext(fname)[0])).rstrip(".tif")
    png.from_array(z, mode="L").save(basedir_StardistWater_FijiMask_RTGPipeline + name.replace("StarWat_FijiMask_",'seg_') + '.png') 
